Берем весь валидированный сет, прогоняем через модельку, обрезаем и сохраняем 
(сохраняем все обрезанное в одну папку + список уже сделанных файлов), это довольно долго делается, так что делал не в колабе, а на персональном компьютере на CPU

In [ ]:
from glob import glob
sc_orders_validated = 'E:\mikiz\segmentation_dataset\photos_securkeys_validated'
all_photos = glob(sc_orders_validated + '\*\*\*', recursive = True)

In [ ]:
import os
#нужны пути до mmdetection и модельки
os.environ['MMDETECTION_DIR'] = 'E:\\develop\\mikiz\\repos\\dataset_prep\\mmdetection'
os.environ['MODEL_PATH'] = 'E:\\develop\\mikiz\\repos\\dataset_prep\\model_detection.pth'

In [ ]:
folder_path = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped'
import cv2
from proc import key_detection
detectron = key_detection()

not_done = []

for photo in all_photos:    
        im = detectron.proc_img(photo)
        if im is not None:
            if cv2.imwrite(f'{folder_path}\\cropped_{os.path.basename(photo)}', im):
                done.append(photo)
            else:
                print(f'error - {photo}')
        else:
            not_done.append(photo)

ресайзим фоточки, длинная сторона до 1024

In [111]:
from glob import glob
path_cropped = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped'
photos_cropped = glob(path_cropped + '\*', recursive = True)

In [114]:
folder_path = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped_resized'
import cv2

for photo in photos_cropped:
    dim = 1024
    img = mmcv.imread(photo)
    y = img.shape[0]
    x = img.shape[1]
    if x > y:
        img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        ratio = y/x
    else:
        ratio = x/y
    
    img = mmcv.imresize(img, (int(ratio*dim), dim))

    cv2.imwrite(f'{folder_path}\\resized_cropped_{os.path.basename(photo)}', img)
        

готовим датасет для разметки под сегментацию, split to train/test/val (60 30 10)

In [16]:
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split

path_resized = 'E:\\mikiz\\segmentation_dataset\\photos_securkeys_validated_cropped_resized'
photos_resized = glob(path_resized + '\*.*', recursive = True)

X_train, X_test = train_test_split(photos_resized, test_size=0.3, random_state=1)

X_train, X_val = train_test_split(X_train, test_size=0.15, random_state=1) #0.7*0.15 ~ 0.1

In [17]:
import shutil

for part in zip([X_train, X_test, X_val], ['train', 'test', 'val']):
    for f_name in part[0]:
        #print(f'{path_resized}\\{part[1]}\\{os.path.basename(f_name)}')
        shutil.copy(f_name, f'{path_resized}\\{part[1]}\\{os.path.basename(f_name)}')